In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
if sys.version_info >= (3,8):
    import pickle
else:
    !pip install pickle5
    import pickle5 as pickle
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     |████████████████████████████████| 256 kB 4.2 MB/s 


In [ ]:
# load data - data can be found here:
# https://drive.google.com/drive/folders/1QJokFJZh1mc6PwskF3M5aWXcUzgzSQ3K?usp=sharing
os.listdir('/content/drive/My Drive/')

with open('/content/drive/My Drive/Colab_Files/emopia/featuresnp.pickle', 'rb') as f:
    featuresnp = pickle.load( f )

with open('/content/drive/My Drive/Colab_Files/emopia/domiantQsnp.pickle', 'rb') as f:
    dominantQsnp = pickle.load( f )

# outputs need to be in range starting from 0
dominantQsnp = dominantQsnp - 1

print('featuresnp.shape: ', featuresnp.shape)
print('dominantQsnp.shape: ', dominantQsnp.shape)

featuresnp.shape:  (1078, 17)
dominantQsnp.shape:  (1078,)


In [ ]:
# scale features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=True, with_std=True)
scaled_features = scaler.fit_transform( featuresnp )

In [ ]:
# split data
from sklearn.model_selection import StratifiedShuffleSplit

stratsplit = StratifiedShuffleSplit( n_splits = 1, test_size = 0.2, random_state = 1)
idxs_generator = stratsplit.split( featuresnp, dominantQsnp )
idxs_list = list (idxs_generator)
train_idxs = idxs_list[0][0]
test_idxs = idxs_list[0][1]

train_input = scaled_features[ train_idxs , : ]
train_output = dominantQsnp[ train_idxs ]

test_input = scaled_features[ test_idxs , : ]
test_output = dominantQsnp[ test_idxs ]

print('train_input.shape: ', train_input.shape)
print('test_input.shape: ', test_input.shape)

train_input.shape:  (862, 17)
test_input.shape:  (216, 17)


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
model = keras.models.Sequential()
model.add(keras.Input(shape=(17,))) # we need to specify input
model.add(keras.layers.Dense(32, activation='tanh'))
model.add(keras.layers.Dense(4, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                576       
                                                                 
 dense_1 (Dense)             (None, 4)                 132       
                                                                 
Total params: 708
Trainable params: 708
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_input, train_output, batch_size=16, epochs=300) # check out batch size and compare it with training instances

Epoch 1/300
54/54 [==============================] - 0s 1ms/step - loss: 1.0066 - accuracy: 0.5870
Epoch 2/300
54/54 [==============================] - 0s 1ms/step - loss: 1.0046 - accuracy: 0.5905
Epoch 3/300
54/54 [==============================] - 0s 1ms/step - loss: 1.0031 - accuracy: 0.5905
Epoch 4/300
54/54 [==============================] - 0s 1ms/step - loss: 1.0013 - accuracy: 0.5940
Epoch 5/300
54/54 [==============================] - 0s 1ms/step - loss: 0.9996 - accuracy: 0.5951
Epoch 6/300
54/54 [==============================] - 0s 2ms/step - loss: 0.9979 - accuracy: 0.5974
Epoch 7/300
54/54 [==============================] - 0s 1ms/step - loss: 0.9963 - accuracy: 0.6056
Epoch 8/300
54/54 [==============================] - 0s 2ms/step - loss: 0.9944 - accuracy: 0.5963
Epoch 9/300
54/54 [==============================] - 0s 1ms/step - loss: 0.9926 - accuracy: 0.5998
Epoch 10/300
54/54 [==============================] - 0s 2ms/step - loss: 0.9914 - accuracy: 0.6044
Epoch 11/

In [ ]:
model.evaluate( test_input, test_output )

7/7 [==============================] - 0s 2ms/step - loss: 1.0247 - accuracy: 0.5972


[1.0247141122817993, 0.5972222089767456]

In [ ]:
# let's make a beefier model
model1 = keras.models.Sequential()
model1.add(keras.Input(shape=(17,))) # we need to specify input
model1.add(keras.layers.Dense(128, activation='tanh'))
model1.add(keras.layers.Dense(4, activation='softmax'))

model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 128)               2304      
                                                                 
 dense_3 (Dense)             (None, 4)                 516       
                                                                 
Total params: 2,820
Trainable params: 2,820
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
history1 = model1.fit(train_input, train_output, epochs=1000)

Epoch 1/1000
27/27 [==============================] - 0s 3ms/step - loss: 0.8419 - accuracy: 0.6647
Epoch 2/1000
27/27 [==============================] - 0s 2ms/step - loss: 0.8410 - accuracy: 0.6659
Epoch 3/1000
27/27 [==============================] - 0s 2ms/step - loss: 0.8406 - accuracy: 0.6601
Epoch 4/1000
27/27 [==============================] - 0s 1ms/step - loss: 0.8394 - accuracy: 0.6647
Epoch 5/1000
27/27 [==============================] - 0s 1ms/step - loss: 0.8392 - accuracy: 0.6601
Epoch 6/1000
27/27 [==============================] - 0s 1ms/step - loss: 0.8388 - accuracy: 0.6659
Epoch 7/1000
27/27 [==============================] - 0s 2ms/step - loss: 0.8377 - accuracy: 0.6613
Epoch 8/1000
27/27 [==============================] - 0s 1ms/step - loss: 0.8375 - accuracy: 0.6659
Epoch 9/1000
27/27 [==============================] - 0s 1ms/step - loss: 0.8368 - accuracy: 0.6682
Epoch 10/1000
27/27 [==============================] - 0s 2ms/step - loss: 0.8369 - accuracy: 0.6705

In [ ]:
model1.evaluate( test_input, test_output )

7/7 [==============================] - 0s 2ms/step - loss: 1.3349 - accuracy: 0.5509


[1.3349363803863525, 0.5509259104728699]